In [42]:
import requests
res = requests.get("http://163.32.98.3/KHHCVS/Main/ClassList.aspx?new=1")
res.close()
res.encoding = res.apparent_encoding

In [43]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text, "lxml")
#print(soup.prettify())

In [44]:
table = soup.select('#DataRow')
from crawlerApi import crawerAPI
#print(table)


In [45]:
import uuid
text= ""
href = 'http://163.32.98.3'
city = "高雄市"
figure = ""
for item in table:
    detail_href1 = 'empty'
    detail_href2 = 'empty'
    
    uniqid = str(uuid.uuid4())[24:]
    text += uniqid + ',' + city + ','
    
    charge = ''
    for idx, cell in enumerate(item.select('td')):
        if idx == 1:
            continue
        elif idx == 3:
            course_name = cell.text.strip().replace(' ','')
            category = crawerAPI.courseClassify(course_name.upper())
            text += course_name + ',' + category + ','
        elif idx == 4:
            try:
                link = cell.find('a').get('href')
                if link[-2:] == '/.':
                    detail_href1 =''
                else:
                    detail_href1 = href + link
            except:
                pass
        elif idx == 5:
            try:
                link = cell.find('a').get('href')
                #判斷網址是否有效
                if link[-2:] == '/.':
                    detail_href2 =''
                else:
                    detail_href2 = href + link
            except:
                pass
        elif idx == 6:
            text += cell.text.strip().replace('/','-')+','
        elif idx == 7:
            weekday = cell.text.strip()
        elif idx == 8:
            text += '每周' + weekday + ' ' +cell.text.strip()+','
        elif idx == 11:
            charge = cell.text.strip()
        elif idx == 12:            
            if cell.text.strip() == '0':
                text += charge + ','
            else:
                text += charge + '  另收雜費:' + cell.text.strip() +','
        elif idx == 13:
            if cell.text.strip() != '':
                text += cell.text.strip().replace('\r\n',' ')+','
            else:
                text += "note_empty,"
        else:
            text += cell.text.strip()+','

    text += detail_href1 + ',' + detail_href2 + ',' + figure
    text += '\n'
print(text)


80f82f2e55f4,高雄市,1,高雄師範大學,家庭之夜幸福學,其他,2016-2-25,每周四 09:00~12:00,40,48,3600,note_empty,http://163.32.98.3/KHHCVS/WebUpFile/Class/3464/105(%e4%b8%80)%e9%ab%98%e5%b8%ab%e5%a4%a7%e5%b8%82%e6%b0%91%e5%ad%b8%e8%8b%91(%e5%ae%b6%e5%ba%ad%e4%b9%8b%e5%a4%9c%e5%b9%b8%e7%a6%8f%e5%ad%b8)%e5%b8%ab%e8%b3%87.pdf,http://163.32.98.3/KHHCVS/WebUpFile/Class/3464/105(%e4%b8%80)%e9%ab%98%e5%b8%ab%e5%a4%a7%e5%b8%82%e6%b0%91%e5%ad%b8%e8%8b%91(%e5%ae%b6%e5%ba%ad%e4%b9%8b%e5%a4%9c%e5%b9%b8%e7%a6%8f%e5%ad%b8)%e8%aa%b2%e7%a8%8b%e8%a1%a8.pdf,
f75b584b2f44,高雄市,2,高雄師範大學,音樂之聲(TheSoundofMusic),其他,2016-2-26,每周五 14:00~17:00,40,48,3600,note_empty,http://163.32.98.3/KHHCVS/WebUpFile/Class/3432/105(%e4%b8%80)%e9%ab%98%e5%b8%ab%e5%a4%a7%e5%b8%82%e6%b0%91%e5%ad%b8%e8%8b%91(%e9%9f%b3%e6%a8%82%e4%b9%8b%e8%81%b2)%e5%b8%ab%e8%b3%87.pdf,http://163.32.98.3/KHHCVS/WebUpFile/Class/3432/105(%e4%b8%80)%e9%ab%98%e5%b8%ab%e5%a4%a7%e5%b8%82%e6%b0%91%e5%ad%b8%e8%8b%91(%e9%9f%b3%e6%a8%82%e4%b9%8b%e8%81%b2)%e8%aa%b2%e7%a8%8b%e8%a1%a8.pdf,
90

In [46]:
title = "ID,city,serial_number,organizer,course_name,category,start_date,classtime,num_of_people,總節數,charge,note,teacher_website,relate_website,figure"

In [47]:
with open('./output_file/' + 'kh_hcvs.csv','w', encoding ='utf8') as outputfile:
    outputfile.write(title+'\n')
    outputfile.write(text)

In [48]:
crawerAPI.csvtojson('kh_hcvs.csv', 'kh_hcvs.json')